In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from src.soporte_variables import url_atrapalo

In [ ]:
res = requests.get(url_atrapalo)
print(res.status_code)
sopa = BeautifulSoup(res.content, "html.parser")
resultados = sopa.find("div", {"id": "resultados_container"})
card_results = resultados.find_all("div", {"class": "card-result"})
imagenes = [f"https://www.atrapalo.com{card.find('img', {'class': 'lazy event-img'}).get('data-original')}" if card.find("img", {'class': 'lazy event-img'}) is not None else "Sin imagen" for card in card_results]
for imagen in imagenes:
    if str(imagen) == "Sin imagen":
        imagenes.remove("Sin imagen")
ratings = [str(rating.getText()).replace("\n", "").strip() for rating in resultados.find_all("span", {"class": "opi-rating"})]
nombres = [str(resultado.find("span").getText()).replace("\n", "").strip() for resultado in resultados.find_all("h2", {"class": "clear nombre"})]
lugares = [resultado.find("span", {"class": ""}) if resultado.find("span", {"class": ""}) is not None else "Sin lugar" for resultado in resultados.find_all("p", {"class": "info"})]
for lugar in lugares:
    if str(lugar) == "Sin lugar":
        lugares.remove("Sin lugar")
#lugares = [lugar.find("a", {"class": "locality GATrackEvent_ubicacion hide-for-small-only"}) if lugar.find("a", {"class": "locality GATrackEvent_ubicacion hide-for-small-only"}) is not None else "None" for lugar in lugares]
lugares
#df = pd.concat([pd.DataFrame(nombres), pd.DataFrame(lugares), pd.DataFrame(ratings), pd.DataFrame(imagenes)], axis=1)
#df

In [ ]:
lugares

In [ ]:
driver = webdriver.Chrome()
url_atrapalo = "https://www.atrapalo.com/actividades/"
driver.get(url_atrapalo)
driver.maximize_window()
sleep(1)
try:
    driver.find_element("css selector", "#soycontrolcookies > div.soycontrolcookies_header > div.soycontrolcookies_buttons > button.btn.btn-default.reject-button").click()
    sleep(1)
    driver.find_element("css selector", "#search_widget > form > input.ui-autocomplete-input").click()
    sleep(1)
    driver.find_element("css selector", "#ecommfinder_searchbox").send_keys(busqueda)
    import random
    lista_url = []
    for n in range(1, int(cantidad)+1):
        dormir = random.choice(range(1,3))
        nombre_url = driver.find_element("css selector", f"#ecommfinder_results > a:nth-child({n})").get_attribute("href")
        lista_url.append(nombre_url)
        sleep(dormir)
    driver.quit()
except:
    print("Error")
    driver.quit()

lista_url